In [ ]:
#Currently I have listed pyspark.sql imports, look for pyspark imports as well
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StructType,StructField, StringType
from pyspark.sql import Row

In [ ]:
spark = SparkSession.builder.appName("TEST").getOrCreate()

In [ ]:
df=spark.read.csv("test.csv",header=True,inferSchema=True)

In [ ]:
#df.show();
#df.printSchema();
#df.columns;df.head(3);
#type(df.select("Experince"))
#df.select("Experince","Name").show();
df.dtypes
#df.describe()
#df.describe().show()
#data2=list("Male") #Will give you ['M', 'a', 'l', 'e']

In [ ]:
df2=spark.createDataFrame([(100,"swaroop"),(200,"Pranavi"),(10,"Krishna")],["Rank","Name"])
df2.dtypes
df2.printSchema()

#select is always better to use than withColumn,obvious reason is too many lines for each addition
# Also some one pointed DAG will result in toomany shuffle operations if you are using it with window functions. 
# see https://stackoverflow.com/questions/59789689/spark-dag-differs-with-withcolumn-vs-select 

# immutable are DataFrames
# you can point to previous df column with df.<column_name> or f.col("<column_name>") function as following 
# lit constant function
# 
df3=df2.select(f.when((df2.Rank == 100) & (df2.Name == "swaroop"),f.lit("swaroop"))\
               .otherwise(f.lit("not swaroop"))).alias("Random_column")


df4=df2.select(f.col("Rank"),f.col("Name"),f.lit("1").alias("What_ever"),\
               f.when((f.col("Rank") == 100) & (f.col("Name") == "swaroop"),\
                    f.lit("this is swaroop")).otherwise(f.lit("not swaroop")).alias("Random_column"))

#********** Select will not add to the DF but only selects what is mentioned and puts it as another DF
#********** WithColumn will add to the exsisting DF

df3.show(truncate=False)
df4.show(truncate=False)

In [ ]:
df4.withColumn("screenTime",f.col("Rank")*-1).show()
df4.withColumn("screenTime",df2.Rank*-1).show()   #Catch is that we can use any dataframe df2.rank or df4.rank
df4.withColumn("newID",f.col("Name").cast("Integer")).show() #cast to change column type

In [ ]:
######## select doesnt validate whether the column is already present in that dataframe,hence duplication  ########
df5=df4.select("*",(f.col("Rank")*10).alias("Rank"))       ##********************IMP**************###
df4.withColumn("Rank",df2.Rank-10).show()       #Value of Rank is changed but no two columns with name "Rank"
df5.show()                                            

In [ ]:
# we see that there are two columns with same name, with different values and now \
#this dataframe is kind of ambiguous, no further select or drop on these columns can \
#be performed, as it leads to AnalysisException.

df5.drop(df5.Rank)        #Will give you Reference 'Rank' is ambiguous, could be: Rank, Rank.
df5.select("Rank")        #Will give you Reference 'Rank' is ambiguous, could be: Rank, Rank

### bottom line is withColumn = "duplication check + select"

In [ ]:
#Add 2 different DF
        #withColumn or select and add specific columns you want
        ## wecan't directly use unionByName or unionAll as we need to allign to have same columns or same no of rows
df6=spark.createDataFrame([\
                         Row(Sex="M"),\
                         Row(Sex="F"),\
                         Row(Sex="F")])
df2_new = df2.select("*",f.lit(None).alias("Sex"))
df6_new = df6.select("*",f.lit(None).alias("Name"),f.lit(None).alias("Rank"))
df6_new.unionByName(df2_new).show()     #unionByName will join according to the column name
df6_new.unionAll(df2_new).show()        #unionAll will join based on no of rows

###Simply add column and update it later with conditions
cols = ['Name',"Rank","Sex"]    
df7=df2.withColumn("Sex", f.lit(None)).select(cols).show()
#or
df7=df2.withColumn("Sex", f.lit(None)).show()


In [ ]:
##join might help here,try when you have time

In [10]:
spark.stop()

In [ ]:
#To learn based on creating,reading,adding DF
    #struct type creating nested or complex columns
        #You can use
        #schema = StructType([StructField("Sex",StringType(),True)])
    #Window function and #partition by column    https://stackoverflow.com/questions/59789689/spark-dag-differs-with-withcolumn-vs-select
    #Anything that is opened in tabs